In [1]:
import toytree
import toyplot
import numpy as np
from ipcoal.smc.smc5 import tree_unch_prob_bt,topo_unch_prob_bt
import pandas as pd

## Use ruler-measured divergence times from the figure to set relative heights, and arbitrarily scale so that the root of the species tree is at 1e6 generations.

In [2]:
ruler_scaler = 250000 # so that the st root will be at 1e6

### Species tree:

In [3]:
# species tree topology
st = toytree.tree('(((a,b),c),d);').mod.edges_extend_tips_to_align()
st.draw(ts='s');

<svg class="toyplot-canvas-Canvas" height="275.0px" id="t94fbe216156c4aab9b7e99b9dbd20292" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 255.0 275.0" width="255.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 a b c d

In [4]:
# set divergence times
wab = 1*ruler_scaler
wabc = 3*ruler_scaler
wabcd = 4*ruler_scaler
st = st.set_node_data(
    "height",
    mapping={4:wab, 
             5: wabc, 
             6: wabcd, 
            }, 
    default=0
)

In [5]:
# set N values
st = st.set_node_data(
    "Ne",
    mapping={0:70000,
             1:70000,
             2:50000,
             3:70000,
             4:100000, 
             5:120000, 
             6:140000, 
            }, 
    default=0
)

### Genealogy:

In [6]:
# genealogy topology
gt = toytree.tree('(((a_1,a_2),(a_3,(b_1,b_2))),(c_1,d_1));')
gt = gt.mod.edges_extend_tips_to_align()
gt.draw(ts='s');

<svg class="toyplot-canvas-Canvas" height="275.0px" id="t07d21a92e2f24ab1bc9461ad9f061d6d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 265.0 275.0" width="265.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 11 12 a_1 a_2 a_3 b_1 b_2 c_1 d_1

In [7]:
# set divergence times
gt = gt.set_node_data(
    "height",
    mapping={7:0.5*ruler_scaler, 
             8: 0.7*ruler_scaler, 
             9: 2.1*ruler_scaler, 
             10: 3.4*ruler_scaler,
             11:4.3*ruler_scaler,
             12:4.8*ruler_scaler,
            }, 
    default=0
)
gt.draw();

<svg class="toyplot-canvas-Canvas" height="275.0px" id="t1049f27b62154a12b3357b7fc38d45f3" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 265.0 275.0" width="265.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> a_1 a_2 a_3 b_1 b_2 c_1 d_1

## Map the genealogy tips to a species tree population.

In [8]:
imap = {'a': ['a_1','a_2','a_3'], 'b': ['b_1','b_2'], 'c': ['c_1'], 'd': ['d_1']}

## Make a sample probabilities plot for one branch.

In [9]:
for idx in [2]:
    node = gt[idx]
    vals = []
    for t in np.linspace(node.height+1,(node.height+node.dist-1),200):
        vals.append(topo_unch_prob_bt(node, t, st, gt, imap))
    
    node = gt[idx]
    trvals = []
    for t in np.linspace(node.height+1,(node.height+node.dist-1),200):
        trvals.append(tree_unch_prob_bt(node, t, st, gt, imap))

canvas = toyplot.Canvas(400, 400) 
axes = canvas.cartesian(grid=(3, 1, 0),ymax=1,ymin=0,margin=25,padding=10)
axes.x.ticks.show = True
axes.x.domain.show = False
axes.y.ticks.show = True
axes.y.domain.show = False
mark = axes.plot(np.linspace(node.height+1,(node.height+node.dist-1),200), 
                 trvals,
                 stroke_width=5,
                 color='#d7191c'
                )

axes = canvas.cartesian(grid=(3, 1, 1),ymax=1,ymin=0,margin=25,padding=10)
axes.x.ticks.show = True
axes.x.domain.show = False
axes.y.ticks.show = True
axes.y.domain.show = False
mark = axes.plot(np.linspace(node.height+1,(node.height+node.dist-1),200), 
                 1-np.array(trvals),
                 stroke_width=5,
                 color='#d7191c'
                )

axes = canvas.cartesian(grid=(3, 1, 2),ymax=1,ymin=0,margin=25,padding=10)
axes.x.ticks.show = True
axes.x.domain.show = False
axes.y.ticks.show = True
axes.y.domain.show = False
mark = axes.plot(np.linspace(node.height+1,(node.height+node.dist-1),200), 
                 1-np.array(vals),
                 stroke_width=5,
                 color='#d7191c'
                )


<svg class="toyplot-canvas-Canvas" height="400.0px" id="ta50bb86b8da540ab8fd3db8037772268" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 400.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 25.00058333333333 79.76698202917784 L 26.53952219430486 79.7211178442112 L 28.07846105527638 79.67258615470335 L 29.617399916247905 79.62123181605772 L 31.156338777219432 79.56689066032058 L 32.69527763819095 79.50938897137422 L 34.23421649916248 79.4485429296066 L 35.773155360134005 79.38415802428246 L 37.312094221105525 79.31602843173756 L 38.85103308207705 79.24393635740795 L 40.38997194304858 79.16765133959105 L 41.928910804020106 79.08692951271283 L 43.467849664991626 79.00151282774598 L 45.00678852596315 78.91112822728678 L 46.54572738693467 78.81548677265364 L 48.0846662479062 78.71428272021693 L 49.623605108877726 78.60719254400729 L 51.162543969849246 78.49387390147774 L 52.70148283082077 78.37396453911384 L 54.24042169179229 78.2470811343928 L 55.779360552763826 78.11281807038992 L 57.318299413735346 77.97074613911516 L 58.857238274706866 77.82041116943422 L 60.39617713567839 77.66133257518847 L 61.93511599664992 77.49300181887196 L 63.47405485762144 77.31488078595473 L 65.01299371859298 77.12640006465472 L 66.5519325795645 76.9269571256601 L 68.090871440536 76.71591439598221 L 69.62981030150755 76.4925972207821 L 71.16874916247906 76.25629170665475 L 72.7076880234506 76.00624243947708 L 74.24662688442211 75.7416500695233 L 75.78556574539364 75.46166875612876 L 77.32450460636517 75.1654034637326 L 78.86344346733668 74.85190710065613 L 80.4023823283082 74.52017749146937 L 81.94132118927975 74.16915417326784 L 83.48026005025127 73.79771500561782 L 85.01919891122279 73.40467258333251 L 86.5581377721943 72.98877044061223 L 88.09707663316583 72.5486790344134 L 89.63601549413735 72.08299149420681 L 91.17495435510888 71.59021912453755 L 92.71389321608041 71.06878664600973 L 94.25283207705192 70.51702715948218 L 95.79177093802346 69.93317681737716 L 97.33070979899497 69.31536918506718 L 98.86964865996651 69.12757045383964 L 100.40858752093804 69.22208763915722 L 101.94752638190955 69.32023499415756 L 103.48646524288107 69.42215194461683 L 105.0254041038526 69.52798327130802 L 106.56434296482414 69.63787931567302 L 108.10328182579563 69.75199619339408 L 109.64222068676717 69.87049601616795 L 111.1811595477387 69.99354712199788 L 112.72009840871021 70.12132431433064 L 114.25903726968176 70.25400911037794 L 115.79797613065327 70.39178999897554 L 117.33691499162481 70.53486270834593 L 118.87585385259632 70.68343048414515 L 120.41479271356785 70.83770437818865 L 121.95373157453936 70.9979035482665 L 123.49267043551089 71.16425556947365 L 125.03160929648243 71.33699675749762 L 126.57054815745393 71.51637250432293 L 128.10948701842548 71.70263762682904 L 129.64842587939697 71.89605672877711 L 131.1873647403685 72.0969045766998 L 132.72630360134005 72.30546649022783 L 134.26524246231156 72.52203874740842 L 135.8041813232831 72.74692900559062 L 137.34312018425462 72.98045673847604 L 138.88205904522613 73.22295368995543 L 140.42099790619764 73.47476434537619 L 141.95993676716918 73.73624642090992 L 143.49887562814072 74.00777137171546 L 145.03781448911224 74.28972491961909 L 146.57675335008378 74.5825076010618 L 148.1156922110553 74.88653533609173 L 149.6546310720268 75.20224001921011 L 151.19356993299834 75.53007013291031 L 152.73250879396988 75.87049138478132 L 154.2714476549414 76.22398736908065 L 155.8103865159129 76.5910602537168 L 157.34932537688445 76.97223149361703 L 158.88826423785596 77.36804257149352 L 160.42720309882748 77.77905576706084 L 161.966141959799 78.20585495579697 L

## Now replicate across three columns for final figure.

In [10]:
# Column 1:

for idx in [2]:
    node = gt[idx]
    vals = []
    for t in np.linspace(node.height+1,(node.height+node.dist-1),200):
        vals.append(topo_unch_prob_bt(node, t, st, gt, imap))
    
    node = gt[idx]
    trvals = []
    for t in np.linspace(node.height+1,(node.height+node.dist-1),200):
        trvals.append(tree_unch_prob_bt(node, t, st, gt, imap))

canvas = toyplot.Canvas(600, 400) 
axes = canvas.cartesian(grid=(3, 3, 0),ymax=1,ymin=0,margin=30,padding=10)
axes.x.ticks.show = True
locations = [0,0.5*ruler_scaler,1*ruler_scaler,2.1*ruler_scaler]
labels = ['','','','']
axes.x.ticks.locator = toyplot.locator.Explicit(locations=locations, labels=labels)
axes.x.domain.show = False
axes.y.ticks.show = True
axes.y.domain.show = False
mark = axes.plot(np.linspace(node.height+1,(node.height+node.dist-1),200), 
                 trvals,
                 stroke_width=5,
                 color='#d7191c'
                )

axes = canvas.cartesian(grid=(3, 3, 3),ymax=1,ymin=0,margin=30,padding=10)
axes.x.ticks.show = True
locations = [0,0.5*ruler_scaler,1*ruler_scaler,2.1*ruler_scaler]
labels = ['','','','']
axes.x.ticks.locator = toyplot.locator.Explicit(locations=locations, labels=labels)
axes.x.domain.show = False
axes.y.ticks.show = True
axes.y.domain.show = False
mark = axes.plot(np.linspace(node.height+1,(node.height+node.dist-1),200), 
                 1-np.array(trvals),
                 stroke_width=5,
                 color='#d7191c'
                )

axes = canvas.cartesian(grid=(3, 3, 6),ymax=1,ymin=0,margin=30,padding=10)
axes.x.ticks.show = True
locations = [0,0.5*ruler_scaler,1*ruler_scaler,2.1*ruler_scaler]
labels = ['0','t<sub>8</sub>','W<sub>ab</sub>','t<sub>9</sub>']
axes.x.ticks.locator = toyplot.locator.Explicit(locations=locations, labels=labels)
axes.x.domain.show = False
axes.y.ticks.show = True
axes.y.domain.show = False
mark = axes.plot(np.linspace(node.height+1,(node.height+node.dist-1),200), 
                 1-np.array(vals),
                 stroke_width=5,
                 color='#d7191c'
                )

# Column 2

for idx in [5]:
    node = gt[idx]
    vals = []
    for t in np.linspace(node.height+1,(node.height+node.dist-1),200):
        vals.append(topo_unch_prob_bt(node, t, st, gt, imap))
    
    node = gt[idx]
    trvals = []
    for t in np.linspace(node.height+1,(node.height+node.dist-1),200):
        trvals.append(tree_unch_prob_bt(node, t, st, gt, imap))

axes = canvas.cartesian(grid=(3, 3, 1),ymax=1,ymin=0,margin=30,padding=10)
axes.x.ticks.show = True
locations = [0,3*ruler_scaler,3.4*ruler_scaler,4*ruler_scaler,4.3*ruler_scaler]
labels = ['','','','','']
axes.x.ticks.locator = toyplot.locator.Explicit(locations=locations, labels=labels)
axes.x.domain.show = False
axes.y.ticks.show = True
axes.y.domain.show = False
mark = axes.plot(np.linspace(node.height+1,(node.height+node.dist-1),200), 
                 trvals,
                 stroke_width=5,
                 color='#d7191c'
                )

axes = canvas.cartesian(grid=(3, 3, 4),ymax=1,ymin=0,margin=30,padding=10)
axes.x.ticks.show = True
locations = [0,3*ruler_scaler,3.4*ruler_scaler,4*ruler_scaler,4.3*ruler_scaler]
labels = ['','','','','']
axes.x.ticks.locator = toyplot.locator.Explicit(locations=locations, labels=labels)
axes.x.domain.show = False
axes.y.ticks.show = True
axes.y.domain.show = False
mark = axes.plot(np.linspace(node.height+1,(node.height+node.dist-1),200), 
                 1-np.array(trvals),
                 stroke_width=5,
                 color='#d7191c'
                )

axes = canvas.cartesian(grid=(3, 3, 7),ymax=1,ymin=0,margin=30,padding=10)
axes.x.ticks.show = True
locations = [0,3*ruler_scaler,3.4*ruler_scaler,4*ruler_scaler,4.3*ruler_scaler]
labels = ['0','W<sub>abc</sub>','t<sub>10</sub>','W<sub>abcd</sub>','t<sub>11</sub>']
axes.x.ticks.locator = toyplot.locator.Explicit(locations=locations, labels=labels)
axes.x.domain.show = False
axes.y.ticks.show = True
axes.y.domain.show = False
mark = axes.plot(np.linspace(node.height+1,(node.height+node.dist-1),200), 
                 1-np.array(vals),
                 stroke_width=5,
                 color='#d7191c'
                )

# Column 3

for idx in [7]:
    node = gt[idx]
    vals = []
    for t in np.linspace(node.height+1,(node.height+node.dist-1),200):
        vals.append(topo_unch_prob_bt(node, t, st, gt, imap))
    
    node = gt[idx]
    trvals = []
    for t in np.linspace(node.height+1,(node.height+node.dist-1),200):
        trvals.append(tree_unch_prob_bt(node, t, st, gt, imap))

axes = canvas.cartesian(grid=(3, 3, 2),ymax=1,ymin=0,margin=30,padding=10)
axes.x.ticks.show = True
locations = [0.5*ruler_scaler,
             1*ruler_scaler,#-0.5*ruler_scaler,
             2.1*ruler_scaler,#-0.5*ruler_scaler,
             3*ruler_scaler,#-0.5*ruler_scaler,
             3.4*ruler_scaler,#-0.5*ruler_scaler
            ]
labels = ['','','','','']
axes.x.ticks.locator = toyplot.locator.Explicit(locations=locations, labels=labels)
axes.x.domain.show = False
axes.y.ticks.show = True
axes.y.domain.show = False
mark = axes.plot(np.linspace(node.height+1,(node.height+node.dist-1),200), 
                 trvals,
                 stroke_width=5,
                 color='#d7191c'
                )

axes = canvas.cartesian(grid=(3, 3, 5),ymax=1,ymin=0,margin=30,padding=10)
axes.x.ticks.show = True
locations = [0.5*ruler_scaler,
             1*ruler_scaler,#-0.5*ruler_scaler,
             2.1*ruler_scaler,#-0.5*ruler_scaler,
             3*ruler_scaler,#-0.5*ruler_scaler,
             3.4*ruler_scaler,#-0.5*ruler_scaler
            ]
labels = ['','','','','']
axes.x.ticks.locator = toyplot.locator.Explicit(locations=locations, labels=labels)
axes.x.domain.show = False
axes.y.ticks.show = True
axes.y.domain.show = False
mark = axes.plot(np.linspace(node.height+1,(node.height+node.dist-1),200), 
                 1-np.array(trvals),
                 stroke_width=5,
                 color='#d7191c'
                )

axes = canvas.cartesian(grid=(3, 3, 8),ymax=1,ymin=0,margin=30,padding=10)
axes.x.ticks.show = True
locations = [0.5*ruler_scaler,
             1*ruler_scaler,#-0.5*ruler_scaler,
             2.1*ruler_scaler,#-0.5*ruler_scaler,
             3*ruler_scaler,#-0.5*ruler_scaler,
             3.4*ruler_scaler,#-0.5*ruler_scaler
            ]
labels = ['t<sub>7</sub>','W<sub>ab</sub>','t<sub>9</sub>','W<sub>abc</sub>','t<sub>10</sub>']
axes.x.ticks.locator = toyplot.locator.Explicit(locations=locations, labels=labels)
axes.x.domain.show = False
axes.y.ticks.show = True
axes.y.domain.show = False
mark = axes.plot(np.linspace(node.height+1,(node.height+node.dist-1),200), 
                 1-np.array(vals),
                 stroke_width=5,
                 color='#d7191c'
                )


<svg class="toyplot-canvas-Canvas" height="400.0px" id="t3e128983fed44790a6142884ac83f7ae" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 400.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 30.00026666666667 78.1949441856765 L 30.703781574539363 78.15458370290585 L 31.40729648241206 78.11187581613896 L 32.11081139028476 78.0666839981308 L 32.814326298157454 78.01886378108212 L 33.51784120603015 77.96826229480932 L 34.22135611390285 77.9147177780538 L 34.924871021775544 77.85805906136856 L 35.62838592964824 77.79810501992905 L 36.331900837520934 77.734663994519 L 37.035415745393635 77.66753317884012 L 37.73893065326634 77.59649797118729 L 38.44244556113903 77.52133128841646 L 39.145960469011726 77.44179284001237 L 39.84947537688442 77.35762835993519 L 40.55299028475712 77.26856879379089 L 41.25650519262982 77.1743294387264 L 41.96002010050251 77.07460903330042 L 42.663535008375206 76.96908879442019 L 43.36704991624791 76.85743139826566 L 44.0705648241206 76.73927990194314 L 44.774079731993304 76.61425660242135 L 45.477594639866 76.48196182910212 L 46.1811095477387 76.34197266616586 L 46.88462445561139 76.19384160060733 L 47.58813936348409 76.03709509164017 L 48.29165427135679 75.87123205689615 L 48.99516917922948 75.69572227058089 L 49.69868408710218 75.51000466846435 L 50.40219899497488 75.31348555428825 L 51.10571390284757 75.10553670185618 L 51.80922881072027 74.88549334673982 L 52.51274371859297 74.65265206118052 L 53.21625862646567 74.40626850539331 L 53.91977353433836 74.1455550480847 L 54.62328844221105 73.8696782485774 L 55.32680335008375 73.57775619249304 L 56.03031825795645 73.2688556724757 L 56.73383316582915 72.94198920494367 L 57.43734807370184 72.59611187333262 L 58.14086298157454 72.23011798773877 L 58.84437788944724 71.84283755028379 L 59.54789279731993 71.43303251490198 L 60.251407705192634 70.99939282959306 L 60.95492261306533 70.54053224848856 L 61.65843752093802 70.05498390034433 L 62.361952428810724 69.5411955992919 L 63.06546733668342 68.99752488285912 L 63.76898224455611 68.83226199937889 L 64.47249715242882 68.91543712245836 L 65.1760120603015 69.00180679485865 L 65.8795269681742 69.09149371126281 L 66.5830418760469 69.18462527875106 L 67.28655678391961 69.28133379779226 L 67.9900716917923 69.38175665018679 L 68.693586599665 69.48603649422779 L 69.39710150753768 69.59432146735814 L 70.10061641541039 69.70676539661096 L 70.80413132328309 69.82352801713259 L 71.50764623115577 69.94477519909847 L 72.21116113902848 70.07067918334442 L 72.91467604690116 70.20141882604773 L 73.61819095477387 70.337179852806 L 74.32170586264657 70.47815512247453 L 75.02522077051927 70.62454490113682 L 75.72873567839196 70.7765571465979 L 76.43225058626466 70.93440780380418 L 77.13576549413736 71.09832111160955 L 77.83928040201005 71.26852992132386 L 78.54279530988275 71.44527602749582 L 79.24631021775544 71.62881051140049 L 79.94982512562814 71.81939409771941 L 80.65334003350084 72.01729752491974 L 81.35685494137354 72.22280192985892 L 82.06036984924623 72.43619924716077 L 82.76388475711893 72.65779262393104 L 83.46739966499163 72.88789685040074 L 84.17091457286433 73.1268388071096 L 84.87442948073702 73.3749579292648 L 85.57794438860972 73.63260668893439 L 86.28145929648242 73.90015109576072 L 86.98497420435511 74.1779712169049 L 87.68848911222781 74.46646171696108 L 88.3920040201005 74.76603241860757 L 89.0955189279732 75.07710888479096 L 89.7990338358459 75.4001330232708 L 90.50254874371859 75.73556371438299 L 91.2060636515913 76.0838774629143 L 91.909578559464 76.44556907501354 L 92.61309346733668 76.82115236110133 L 93.31660837520938 77.21116086577685 L 94.02012